In [ ]:
import matplotlib
%matplotlib widget

import numpy as np
from split_dataset import SplitDataset
from pathlib import Path
import flammkuchen as fl
from tifffile import imread
import matplotlib.pyplot as plt 
from fimpylab.core.twop_experiment import TwoPExperiment
from bouterin.plots.stimulus_log_plot import get_paint_function

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
master = Path(r"\\Funes\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\2p\rf")
fish_list = list(master.glob("*f[0-9]*"))
fish_dir = fish_list[2]
# 0, 1, 2, 3, 4

In [ ]:
traces = fl.load(fish_dir / "traces.h5")['traces'][:, 0:-2]
fs = 3
t = np.arange(np.shape(traces)[1]) / fs
num_traces, len_rec = np.shape(traces)
print("num_traces: ", num_traces)
print("len_rec: ", len_rec)
new_len_rec = int(len_rec/3)

In [ ]:
norm_traces = np.copy(traces)
norm_traces=norm_traces.T# need to transpose it since the functions work like that 
sd=np.nanstd(norm_traces)
mean=np.nanmean(norm_traces)
norm_traces=norm_traces-mean #numerator in the formula for z-score 
norm_traces=norm_traces/sd
traces=norm_traces.T

In [ ]:
trial_traces = np.zeros((3, num_traces, new_len_rec))
for i in range(3):
    t1 = i * new_len_rec
    t2 = t1 + new_len_rec
    trial_traces[i] = traces[:, t1:t2]
avg_traces = np.nanmean(trial_traces, 0)

In [ ]:
num_rois = np.shape(avg_traces)[0]
print(num_rois)

In [ ]:
exp = TwoPExperiment(path=fish_dir)
num_subtrials = 16
num_rep = 3
num_trials = num_rep * num_subtrials

stimulus_log = exp.load_session_log(log_name='stimulus_log', session_idx=0)
stim_value, t_values = get_paint_function(stimulus_log, 'E0040_motions_cardinal')
stim_value = stim_value / 255
num_stim = np.shape(stim_value)[0]

In [ ]:
def show_trials(x, y, z):
    ax1.cla()
    for i in range(17):
        ax1.axvspan(
            t_values[i, 0],
            t_values[i, 1],
            facecolor=[stim_value[i, 0], stim_value[i, 1], stim_value[i, 2]],
            alpha=0.5,
        )
    trace_ind = x + y + z
    t_avg = t[0:new_len_rec]
    for i in range(3):
        ax1.plot(t_avg, trial_traces[i, trace_ind], c='gray')
    
    trace = avg_traces[x]
    line = ax1.plot(t_avg, avg_traces[trace_ind], c='k')    
    ax1.set_ylim(np.min(avg_traces[:,trace_ind]) - 2, np.max(trial_traces[:,trace_ind]) + 2)

In [ ]:
fig1, ax1 = plt.subplots(figsize=(13,5))
for i in range(17):
    ax1.axvspan(
        t_values[i, 0],
        t_values[i, 1],
        facecolor=[stim_value[i, 0], stim_value[i, 1], stim_value[i, 2]],
        alpha=0.5,
    )

interact(show_trials, x=(0, num_rois-1, 100), y=(0, 90, 10), z=(0, 9, 1))

In [ ]:
fig_save, ax_save = plt.subplots(figsize=(13,5))
for i in range(17):
    ax_save.axvspan(
        t_values[i, 0],
        t_values[i, 1],
        facecolor=[stim_value[i, 0], stim_value[i, 1], stim_value[i, 2]],
        alpha=0.5,
    )
    
t_avg = t[0:new_len_rec]
ind_trace = 231

for i in range(3):
    ax_save.plot(t_avg, trial_traces[i, ind_trace], c='gray')
ax_save.plot(t_avg, avg_traces[ind_trace], c='k')
ax_save.set_ylim(-2,2)
plt.show()

file_name = "roi" + str(ind_trace) + ".jpg"
fig_save.savefig(fish_dir / file_name, dpi=300)

In [ ]:
nice_inds = [1, 2, 3, 5, 7, 12, 14, 15, 16, 19, 27, 33, 36, 45, 48, 66, 67, 68, 74, 79, 80, 89, 90, 92, 97, 100, 111, 112, 114, 129, 132, 138, 140, 156, 165, 168, 169]
reliable_inds = [9, 25, 46, 78, 101, 113, 143, 159, 176]
ihnhib_resp_inds = [51, 84, 85, 134]

In [ ]:
len(nice_inds)

In [ ]:
a = np.array([[1, 2, 5, 1, 9, 5, 6], [8, 3, 4, 3, 2, 1, 7], [2, 1, 6, 3, 4, 9, 0]])
print(a)
b = [1, 0, 2]
c = a[b, :]
print(c)

In [ ]:
max_vals = np.zeros((num_traces, 2, 8))

for i in range(8):
    t1 = (i * 15) + 5
    t2 = t1 + (fs * 10)
    tmp_traces = avg_traces[:, t1:t2]
    tmp_max = np.amax(tmp_traces, axis=1)
    max_vals[:, 0, i] = tmp_max
    
    t1 = ((8-i) * 15) + 5
    t2 = t1 + (fs * 10)
    tmp_traces = avg_traces[:, t1:t2]
    tmp_max = np.amax(tmp_traces, axis=1)
    max_vals[:, 1, 7-i] = tmp_max

In [ ]:
max_diff = max_vals[:, 1, :] - max_vals[:, 0, :]
fig2, ax2 = plt.subplots(figsize=(13,5))
ax2.imshow(max_diff, extent=[0,  1000, 0, 500])#, vmin=-20, vmax=1500)
print(max_diff)

In [ ]:
roi_map = fl.load(fish_dir / "merged_rois.h5")['stack'][0:12,:,:]
fig3, ax3 = plt.subplots(3, 5, figsize=(12, 12))
num_rois = num_traces
num_planes = np.shape(roi_map)[0]

for i in range(num_planes):
    r = i // 5
    c = np.mod(i, 5)
    
    roi_layer = roi_map[i]
    roi_layer = np.ma.masked_where(roi_layer < 1, roi_layer)
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    ax3[r, c].imshow(roi_layer, cmap="rainbow")
    
plt.show()

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
roi_map_max_diff = np.zeros_like(roi_map)
roi_map_max_diff_pos = np.zeros_like(roi_map)
roi_map_max_diff_neg = np.zeros_like(roi_map)

roi_map_max_diff_pos_val = np.zeros_like(roi_map)
roi_map_max_diff_neg_val = np.zeros_like(roi_map)

roi_map_max_0 = np.zeros_like(roi_map)
roi_map_max_1 = np.zeros_like(roi_map_max_0)
roi_map_max_2 = np.zeros_like(roi_map_max_0)
roi_map_max_3 = np.zeros_like(roi_map_max_0)
roi_map_max_4 = np.zeros_like(roi_map_max_0)
roi_map_max_5 = np.zeros_like(roi_map_max_0)
roi_map_max_6 = np.zeros_like(roi_map_max_0)
roi_map_max_7 = np.zeros_like(roi_map_max_0)

max_diff_filtered = np.copy(max_diff)
max_diff_filtered[np.where(abs(max_diff_filtered) < 200)[0]] = 0
tmp_max = np.argmax(abs(max_diff), axis=1)
tmp_max_pos = np.argmax(max_diff, axis=1)
tmp_max_neg = np.argmax(max_diff * -1, axis=1)

print(tmp_max)
tmp_max_val = np.amax(abs(max_diff), axis=1)
tmp_max_pos_val = np.amax(max_diff, axis=1)
tmp_max_neg_val = np.amax(max_diff * -1, axis=1)

#tmp_max[np.where(tmp_max_val < 200)[0]] = -1
#tmp_max_pos[np.where(tmp_max_pos_val < 200)[0]] = -1
#tmp_max_neg[np.where(tmp_max_neg_val < 200)[0]] = -1
for i in range(0, num_rois):
    
    roi_map_max_diff[np.where(roi_map == (i + 1))] = tmp_max[i] + 1
    roi_map_max_diff_pos[np.where(roi_map == (i + 1))] = tmp_max_pos[i] + 1
    roi_map_max_diff_neg[np.where(roi_map == (i + 1))] = tmp_max_neg[i] + 1
    
    roi_map_max_diff_pos_val[np.where(roi_map == (i + 1))] = tmp_max_pos_val[i]
    roi_map_max_diff_neg_val[np.where(roi_map == (i + 1))] = tmp_max_neg_val[i]
    
    roi_map_max_0[np.where(roi_map == (i + 1))] = max_diff[i, 0]
    roi_map_max_1[np.where(roi_map == (i + 1))] = max_diff[i, 1]
    roi_map_max_2[np.where(roi_map == (i + 1))] = max_diff[i, 2]
    roi_map_max_3[np.where(roi_map == (i + 1))] = max_diff[i, 3]
    roi_map_max_4[np.where(roi_map == (i + 1))] = max_diff[i, 4]
    roi_map_max_5[np.where(roi_map == (i + 1))] = max_diff[i, 5]
    roi_map_max_6[np.where(roi_map == (i + 1))] = max_diff[i, 6]
    roi_map_max_7[np.where(roi_map == (i + 1))] = max_diff[i, 7]


In [ ]:
ind_sort = np.argsort(tmp_max_val)
sorted_traces = avg_traces[ind_sort]
np.shape(sorted_traces)
fig10, ax10 = plt.subplots(1, 1, figsize=(12, 12))
ax10.imshow(sorted_traces,extent=[0,  1000, 0, 1000])
print(ind_sort)
fig10.savefig(str(fish_dir/'traces_sorted_max_diff_abs.jpg'), dpi=300)

In [ ]:
fig4, ax4 = plt.subplots(3, 4, figsize=(12, 12))
color_list = ["lightskyblue", "turquoise", "lightgreen", "khaki", "gold","pink", "plum", "violet"]
cm_roi = LinearSegmentedColormap.from_list("my_list", color_list, N=8)

for i in range(num_planes):
    r = i // 4
    c = np.mod(i, 4)
    
    roi_layer = roi_map_max_diff[i]
    roi_layer[0, 1:9] = [1, 2, 3, 4, 5, 6, 7, 8]
    roi_layer = np.ma.masked_where(roi_layer < 1, roi_layer)
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    #ax4[r, c].imshow(roi_layer, cmap="rainbow")
    ax4[r, c].imshow(roi_layer, cmap=cm_roi)
    ax4[r, c].axis('off')
    ax4[r, c].set_title("z" + str(i))
    #print(roi_layer)

    
plt.show()
fig4.savefig(str(fish_dir/'rois_direction_of_max_abs_diff.jpg'))

In [ ]:
fig6, ax6 = plt.subplots(3, 4, figsize=(12, 12))
color_list = ["lightskyblue", "turquoise", "lightgreen", "khaki", "gold","pink", "plum", "violet"]
cm_roi = LinearSegmentedColormap.from_list("my_list", color_list, N=8)

for i in range(num_planes):
    r = i // 4
    c = np.mod(i, 4)
    
    roi_layer = roi_map_max_diff_pos[i]
    roi_layer[0, 1:9] = [1, 2, 3, 4, 5, 6, 7, 8]
    roi_layer = np.ma.masked_where(roi_layer < 1, roi_layer)
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    #ax4[r, c].imshow(roi_layer, cmap="rainbow")
    ax6[r, c].imshow(roi_layer, cmap=cm_roi)
    ax6[r, c].axis('off')
    ax6[r, c].set_title("z" + str(i))
    #print(roi_layer)

    
plt.show()
fig6.savefig(str(fish_dir/'rois_direction_of_max_diff_pos.jpg'))

In [ ]:
fig7, ax7 = plt.subplots(3, 4, figsize=(12, 12))
color_list = ["lightskyblue", "turquoise", "lightgreen", "khaki", "gold","pink", "plum", "violet"]
cm_roi = LinearSegmentedColormap.from_list("my_list", color_list, N=8)

for i in range(num_planes):
    r = i // 4
    c = np.mod(i, 4)
    
    roi_layer = roi_map_max_diff_neg[i]
    roi_layer[0, 1:9] = [1, 2, 3, 4, 5, 6, 7, 8]
    roi_layer = np.ma.masked_where(roi_layer < 1, roi_layer)
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    #ax4[r, c].imshow(roi_layer, cmap="rainbow")
    ax7[r, c].imshow(roi_layer, cmap=cm_roi)
    ax7[r, c].axis('off')
    ax7[r, c].set_title("z" + str(i))
    #print(roi_layer)

    
plt.show()
fig7.savefig(str(fish_dir/'rois_direction_of_max_abs_diff_neg.jpg'))

In [ ]:
fig8, ax8 = plt.subplots(3, 4, figsize=(12, 12))
color_list = ["lightskyblue", "turquoise", "lightgreen", "khaki", "gold","pink", "plum", "violet"]
cm_roi = LinearSegmentedColormap.from_list("my_list", color_list, N=8)

for i in range(num_planes):
    r = i // 4
    c = np.mod(i, 4)
    
    roi_layer = roi_map_max_diff_pos_val[i]
    roi_layer[0, 1:3] = [-5, 5]
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    ax8[r, c].imshow(roi_layer, cmap="bwr")
    ax8[r, c].axis('off')
    ax8[r, c].set_title("z" + str(i))

    
plt.show()
fig8.savefig(str(fish_dir/'rois_max_pos_diff_value.jpg'))

In [ ]:
fig9, ax9 = plt.subplots(3, 4, figsize=(12, 12))

for i in range(num_planes):
    r = i // 4
    c = np.mod(i, 4)
    
    roi_layer = roi_map_max_diff_neg_val[i]
    roi_layer[0, 1:3] = [-5, 5]
    #roi_layer = np.ma.masked_where(roi_layer < 1, roi_layer)
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    #ax4[r, c].imshow(roi_layer, cmap="rainbow")
    ax9[r, c].imshow(roi_layer, cmap="bwr")
    ax9[r, c].axis('off')
    ax9[r, c].set_title("z" + str(i))
    #print(roi_layer)

    
plt.show()
fig9.savefig(str(fish_dir/'rois_max_neg_diff_value.jpg'))

In [ ]:
fig5, ax5 = plt.subplots(3, 5, figsize=(12, 12))

for i in range(num_planes):
    r = i // 5
    c = np.mod(i, 5)
    
    roi_layer = roi_map_max_5[i]
    roi_layer = np.rot90(roi_layer, k=1, axes=(1, 0))

    ax5[r, c].imshow(roi_layer, cmap="bwr")#rainbow")
    ax5[r, c].axis('off')
    #print(roi_layer)

    
plt.show()